### N-Hits

In [1]:
import pandas as pd
from darts import TimeSeries
from darts.models import NBEATSModel, TCNModel  # You can try other models like TFTModel, TransformerModel
from darts.dataprocessing.transformers import Scaler
from sklearn.preprocessing import MinMaxScaler
import torch
import numpy as np
from darts.models import NHiTSModel


/home/suraj-desk/projects/time_series/env/lib/python3.12/site-packages/fs/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)  # type: ignore
/home/suraj-desk/projects/time_series/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("preprocessed_dataset.csv", parse_dates=["timestamp"])
df.set_index("timestamp", inplace=True)
df = df.fillna(method='ffill', axis=1).fillna(0)

/tmp/ipykernel_445153/3835831487.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill', axis=1).fillna(0)


In [3]:
# Normalize the data (optional but recommended)
scaler_target = Scaler(MinMaxScaler())
scaler_covariates = Scaler(MinMaxScaler())

# Convert to Darts TimeSeries
series_target = TimeSeries.from_dataframe(df, value_cols=["requests"]).astype(np.float32)
series_covariates = TimeSeries.from_dataframe(df, value_cols=["memory", "cpu"]).astype(np.float32)

# Normalize
series_target = scaler_target.fit_transform(series_target)
series_covariates = scaler_covariates.fit_transform(series_covariates)


In [4]:
# Define train-test split
train_size = 0.8  # 80% training, 20% testing
train_target, test_target = series_target.split_after(train_size)
train_covariates, test_covariates = series_covariates.split_after(train_size)


In [5]:
train_covariates.shape

(5760, 2, 1)

In [6]:
window = 11
prediction_length = 2 # one step forecast
eps = 50

In [7]:
model = NHiTSModel(
    input_chunk_length=6,
    output_chunk_length=6,
    num_blocks=2,
    n_epochs=5,

)

In [8]:
model.fit(train_target, past_covariates=train_covariates, verbose=True)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A2000 12GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | stacks          | ModuleList       | 1.7 M  | train
-------------------------

Epoch 4: 100%|██████████| 180/180 [00:04<00:00, 38.12it/s, train_loss=0.000121]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 180/180 [00:04<00:00, 38.10it/s, train_loss=0.000121]


NHiTSModel(output_chunk_shift=0, num_stacks=3, num_blocks=2, num_layers=2, layer_widths=512, pooling_kernel_sizes=None, n_freq_downsample=None, dropout=0.1, activation=ReLU, MaxPool1d=True, input_chunk_length=6, output_chunk_length=6, n_epochs=5)

In [9]:
test_covariates_extended = series_covariates[-(len(test_target) + model.input_chunk_length):]

#nHiTS model predcition
nhits_pred_series = model.predict(n=len(test_target),past_covariates=test_covariates_extended)

`predict()` was called with `n > output_chunk_length`: using auto-regression to forecast the values after `output_chunk_length` points. The model will access `(n - output_chunk_length)` future values of your `past_covariates` (relative to the first predicted time step). To hide this warning, set `show_warnings=False`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  2.07it/s]


In [18]:
nhits_pred_original = scaler_target.inverse_transform(nhits_pred_series)
test_target_original = scaler_target.inverse_transform(test_target)

In [24]:
from darts.metrics import mse, rmse, mae, r2_score, mape

def evaluate_forecast(true_series, pred_series):
    results = {
        "MSE": mse(true_series, pred_series),
        "RMSE": rmse(true_series, pred_series),
        "MAE": mae(true_series, pred_series),
        "MAPE": mape(true_series, pred_series),
        "R2": r2_score(true_series, pred_series)
    }
    return results


In [25]:
results = evaluate_forecast(test_target_original, nhits_pred_original)

In [26]:
results

{'MSE': np.float32(157703.67),
 'RMSE': np.float32(397.1192),
 'MAE': np.float32(135.73431),
 'MAPE': np.float32(388.96042),
 'R2': np.float32(-0.012715459)}